In [1]:
import nasdaqdatalink
data = nasdaqdatalink.get('OPT/SPX')

In [5]:
import numpy as np
import pandas as pd

In [37]:
row=data.iloc[0].to_dict()
row
out_terms=np.array([1,30,60.90,120,540])

In [54]:

#row=data[0].to_dict()
row=data.iloc[0].to_dict()
#compute vols
import py_vollib.black_scholes
import py_vollib_vectorized
def process_orats_vols_row(row):
    terms={'iv30':30,
        'iv60':60,
        'iv90':90}

    for i in range(1,5):
        terms['m'+str(i)+'atmiv']=row["m"+str(i)+"dtex"]

    sorted_arr=sorted(terms.items(), key=lambda x: x[1])
    terms=np.array([x[1] for x in sorted_arr])
    vols=np.array([row[x[0]] for x in sorted_arr])

    weights30=np.sqrt(30)/np.sqrt(np.maximum(terms,30))
    weights_inf=1-weights30
    orats_interp_slopes=weights30*row["slope"]+(1-weights30)*row["slope_inf"]
    orats_interp_derivs=weights30*row["deriv"]+(1-weights30)*row["deriv_inf"]
    texp=terms/365.25

    terms
    orats_interp_slopes



    option_price90=py_vollib_vectorized.models.vectorized_black_scholes_merton('c', 100,95,texp, 0, vols,0,return_as='array')
    option_greeks90 = py_vollib_vectorized.get_all_greeks('c', 100,95,texp, 0, vols,0,model='black_scholes_merton', return_as='dict')
    option_price110=py_vollib_vectorized.models.vectorized_black_scholes_merton('c', 100,105,texp, 0, vols,0,return_as='array')
    option_greeks110 = py_vollib_vectorized.get_all_greeks('c', 100,105,texp, 0, vols/100,0,model='black_scholes_merton', return_as='dict')
    delta90=np.array(option_greeks90['delta'])
    delta110=np.array(option_greeks110['delta'])
    vols
    vols90=vols*(1+(orats_interp_slopes/1000+(orats_interp_derivs/1000*(delta90*100-50)/2))*(delta90*100-50))
    vols110=vols*(1+(orats_interp_slopes/1000+(orats_interp_derivs/1000*(delta110*100-50)/2))*(delta110*100-50))
    log_slopes=(vols110-vols90)/(np.log(1.05/.95)*10)*np.sqrt(texp)
    log_slopes
    out_slopes=np.interp(out_terms,terms,log_slopes)
    out_vols=np.interp(out_terms,terms,vols)
    result={}
    #print("row={}".format(row))
    #result["date"]=row.index[0]
    for i in range(len(out_terms)):
        result['iv'+str(int(out_terms[i]))]=out_vols[i]
        result['slope'+str(int(out_terms[i]))]=out_slopes[i]
    return result

process_orats_vols_row(row)

{'iv1': 15.12,
 'slope1': -1.4918641388984977,
 'iv30': 16.08,
 'slope30': -2.586797619969007,
 'iv60': 17.78664705882353,
 'slope60': -4.082762370442339,
 'iv120': 19.47,
 'slope120': -5.640734862848665,
 'iv540': 19.47,
 'slope540': -5.640734862848665}

In [55]:
data
out_rows=[process_orats_vols_row(row) for row in data.to_dict(orient='records')]
#out_rows

data.to_dict(orient='records')

[{'stockpx': 1132.99,
  'iv30': 16.08,
  'iv60': 17.73,
  'iv90': 19.63,
  'm1atmiv': 15.12,
  'm1dtex': 14.0,
  'm2atmiv': 16.83,
  'm2dtex': 49.0,
  'm3atmiv': 18.8,
  'm3dtex': 77.0,
  'm4atmiv': 19.47,
  'm4dtex': 105.0,
  'slope': 5.82029,
  'deriv': 0.1246,
  'slope_inf': 6.62185,
  'deriv_inf': 0.1089,
  '10dclsHV': 10.53,
  '20dclsHV': 10.82,
  '60dclsHV': 16.0,
  '120dclsHV': 16.37,
  '252dclsHV': 27.1,
  '10dORHV': 10.71,
  '20dORHV': 11.27,
  '60dORHV': 16.03,
  '120dORHV': 16.72,
  '252dORHV': 27.18},
 {'stockpx': 1136.52,
  'iv30': 15.86,
  'iv60': 17.18,
  'iv90': 19.04,
  'm1atmiv': 15.26,
  'm1dtex': 13.0,
  'm2atmiv': 16.27,
  'm2dtex': 48.0,
  'm3atmiv': 18.1,
  'm3dtex': 76.0,
  'm4atmiv': 18.87,
  'm4dtex': 104.0,
  'slope': 5.63972,
  'deriv': 0.1245,
  'slope_inf': 6.58748,
  'deriv_inf': 0.1098,
  '10dclsHV': 10.45,
  '20dclsHV': 10.74,
  '60dclsHV': 15.95,
  '120dclsHV': 15.86,
  '252dclsHV': 27.09,
  '10dORHV': 10.42,
  '20dORHV': 10.44,
  '60dORHV': 15.98,
  '